**Histórico dos dados dos fundos de investimentos**
* https://dados.cvm.gov.br/dataset/fi-doc-cda

* Observação: muitas vezes, os arquivos excel do site acima não se atualizam rapidamente. Para ver a posição mais atualizada entrar no site:
* https://conteudo.cvm.gov.br/menu/regulados/fundos/consultas/fundos.html
---
O arquivo zipado da CVM contém vários arquivos excel. Os arquivos que eu tenho interesse são:

* cda_fi_BLC_1: arquivo que contém os 'Títulos Públicos';
* cda_fi_BLC_2: arquivo que contém as 'Cotas de Fundos';
* cda_fi_BLC_4: arquivo que contém as 'Ações', 'BDRs', 'Opções', 'Mercado futuro', 'Debêntures', 'Obrigações por ações' (posição vendida);
* cda_fi_BLC_7: arquivo que contém os 'Investimentos no Exterior';
* cda_fi_BLC_8: arquivo que contém os 'Investimentos no Exterior';
* cda_fi_PL: arquivo que contém o PL dos fundos de investimentos.

**Qual é o objetivo desse notebook?**
* Criar um arquivo excel para cada fundo de investimento, onde cada classe de ativo estará em uma aba diferente.



In [34]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import zipfile

## Extraindo os arquivos zipados

In [218]:
# Mês e ano escolhido
mes = '05'
ano = '2023'

# Nome do arquivo zipad
zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm_zip//cda_fi_{ano}{mes}.zip'

# # Visualizando a lista dos nomes dos arquivos dentro do arquivo zipado
# with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
#     print("Arquivos dentro do arquivo zipado:")
#     print(zip_ref.namelist())

# Lista dos nomes dos arquivos que você deseja extrair
arquivos_a_extrair = [
    f'cda_fi_BLC_1_{ano}{mes}.csv', 
    f'cda_fi_BLC_2_{ano}{mes}.csv', 
    f'cda_fi_BLC_4_{ano}{mes}.csv', 
    f'cda_fi_BLC_7_{ano}{mes}.csv', 
    f'cda_fi_BLC_8_{ano}{mes}.csv',
    f'cda_fi_PL_{ano}{mes}.csv'
]

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    # Extrai os arquivos selecionados para um diretório específico
    for arquivo in arquivos_a_extrair:
        zip_ref.extract(arquivo, f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}')


## Funções

In [200]:
def open_cda_1(path) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_1'.

    Parameters:
    path (str): caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_1'

    """
    # Lendo o arquivo
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'TP_TITPUB', 'DT_VENC']]

    # Mesclando as colunas 'TP_TITPUB' e 'DT_VENC' em apenas em uma coluna
    df['TP_TITPUB'] = df['TP_TITPUB'] + ' ' + df['DT_VENC']

    # Removendo a coluna 'DT_VENC'
    df = df.drop('DT_VENC', axis=1)

    # Renomeando a coluna 'TP_TITPUB' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_2/4/7/8 para fazer depois juntar os dfs
    df = df.rename(columns={'TP_TITPUB':'CD_ATIVO'})

    # Transformando os dtypes das colunas.
    df['TP_FUNDO'] = df['TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df['DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df['TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df['TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df['VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df['CD_ATIVO'].astype(str)

    return df


def open_cda_2(path) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_2'.

    Parameters:
    path (str): caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_2'

    """
    # Lendo o arquivo. Adicionei o 'low_memory=False' para não dar o aviso -> DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1', low_memory=False)

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'NM_FUNDO_COTA']]

    # Renomeando a coluna 'NM_FUNDO_COTA' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4/7/8 para fazer depois juntar os dfs
    df = df.rename(columns={'NM_FUNDO_COTA':'CD_ATIVO'})

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df['TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df['DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df['TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df['TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df['VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df['CD_ATIVO'].astype(str)

    return df


def open_cda_4(path) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_4'.

    Parameters:
    path (str): caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_4'

    """
    # Lendo o arquivo
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'CD_ATIVO']]

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    return df


def open_cda_7(path) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_7'.

    Parameters:
    path (str): caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_7'

    """
    # Lendo o arquivo
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas.
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'EMISSOR']]

    # Renomeando a coluna 'EMISSOR' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4 p/ fazer depois juntar os dfs.
    df.rename(columns={"EMISSOR": "CD_ATIVO"}, inplace=True)

    # Transformando os dtypes das colunas.
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    return df


def open_cda_8(path) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_BLC_8'.

    Parameters:
    path (str): caminho do arquivo.

    Returns:
    Dataframe do arquivo 'cda_fi_BLC_8'

    """
    # Lendo o arquivo
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando as principais colunas
    df = df[['TP_FUNDO', 'CNPJ_FUNDO', 'DENOM_SOCIAL','DT_COMPTC' , 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'DS_ATIVO']]

    # Renomeando a coluna 'DS_ATIVO' p/ 'CD_ATIVO'. Assim fica igual ao df do arquivo cda_fi_BLC_4 p/ fazer depois juntar os dfs
    df.rename(columns={"DS_ATIVO": "CD_ATIVO"}, inplace=True)

    # Transformando os dtypes das colunas
    df['TP_FUNDO'] = df.loc[:, 'TP_FUNDO'].astype(str)
    df['CNPJ_FUNDO'] = df.loc[:, 'CNPJ_FUNDO'].astype(str)
    df['DENOM_SOCIAL'] = df.loc[:, 'DENOM_SOCIAL'].astype(str)
    df['DT_COMPTC'] = pd.to_datetime(df['DT_COMPTC'])
    df['TP_APLIC'] = df.loc[:, 'TP_APLIC'].astype(str)
    df['TP_ATIVO'] = df.loc[:, 'TP_ATIVO'].astype(str)
    df['VL_MERC_POS_FINAL'] = df.loc[:, 'VL_MERC_POS_FINAL'].astype(float)
    df['CD_ATIVO'] = df.loc[:, 'CD_ATIVO'].astype(str)

    # Selecionando apenas o ativo 'BDR', porque neste arquivo também possui um ativo chamado 'Títulos Públicos', mas não é o principal 'Títulos Públicos', que está no 'cda_fi_BLC_1'
    filt_bdr = (df['TP_APLIC'] == 'Brazilian Depository Receipt - BDR')
    df = df.loc[filt_bdr].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    
    return df


def pl_fundo(path: str, cnpj: str) -> DataFrame:
    """
    Formatando o arquivo 'cda_fi_PL'.
    
    Paramenters:
    path (str): caminho do arquivo.
    cnpj (str): cnpj do fundo de investimento que você está procurando.

    Returns:
    Dataframe com o valor do patrimônio líquido do fundo de investimentos específico.

    """
    # Lendo o arquivo
    df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

    # Selecionando apenas os 'Fundos de Investimentos'
    filt_fi = df['TP_FUNDO'] == 'FI'
    df = df.loc[filt_fi]

    # Selecionando o fundo de investimentos específicos
    filt_cnpj = df['CNPJ_FUNDO'] == cnpj
    fundo_espec = df.loc[filt_cnpj]

    # # Transformando os dtypes da coluna
    # fundo_espec['VL_PATRIM_LIQ'] = fundo_espec.loc[:, 'VL_PATRIM_LIQ'].astype(float)

    return fundo_espec['VL_PATRIM_LIQ']


def fundo_cnpj(cnpj: str) -> DataFrame:
    """
    Separando o df do fundo de investimentos em várias categorias.

    Parameters:
    cnpj (str): cnpj do fundo de investimento que você está procurando.

    Returns:
    Vários dataframes de categorias diferentes: ações, BDRs, investimentos no exterior, cotas de fundos e títulos públicos.
    
    """
    # Lendo o df concatenado
    filt_cnpj = df_ativos['CNPJ_FUNDO'] == cnpj
    fundo_espec = df_ativos.loc[filt_cnpj]

    # Ações
    filt_acoes = (fundo_espec['TP_APLIC'] == 'Ações')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_acoes = fundo_espec.loc[filt_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_acao = lambda x: (x / df_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_acoes['PORCENTAGEM'] = list(map(porcentagem_acao, df_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_acoes = df_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # BDRs
    filt_bdr = (fundo_espec['TP_APLIC'] == 'Brazilian Depository Receipt - BDR')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_bdr = fundo_espec.loc[filt_bdr].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_bdr = lambda x: (x / df_bdr['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_bdr['PORCENTAGEM'] = list(map(porcentagem_bdr, df_bdr['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_bdr =  df_bdr.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Investimentos Exterior
    filt_exterior = (fundo_espec['TP_APLIC'] == 'Investimento no Exterior')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_exterior = fundo_espec.loc[filt_exterior].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_exterior = lambda x: (x / df_exterior['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_exterior['PORCENTAGEM'] = list(map(porcentagem_exterior, df_exterior['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_exterior = df_exterior.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Cotas de Fundos
    filt_cotas_fundos = (fundo_espec['TP_APLIC'] == 'Cotas de Fundos')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_cotas_fundos = fundo_espec.loc[filt_cotas_fundos].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada cota de fundo
    porcentagem_cotas = lambda x: (x / df_cotas_fundos['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_cotas_fundos['PORCENTAGEM'] = list(map(porcentagem_cotas, df_cotas_fundos['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_cotas_fundos = df_cotas_fundos.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Títulos públicos
    filt_titulos_pub = (fundo_espec['TP_APLIC'] == 'Títulos Públicos')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_titulos_pub = fundo_espec.loc[filt_titulos_pub].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada título público
    porcentagem_titulos = lambda x: (x / df_titulos_pub['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_titulos_pub['PORCENTAGEM'] = list(map(porcentagem_titulos, df_titulos_pub['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_titulos_pub = df_titulos_pub.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    # Obrigações por ações e outros TVM recebidos em empréstimo
    filt_vendido_acoes = (fundo_espec['TP_APLIC'] == 'Obrigações por ações e outros TVM recebidos em empréstimo')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_vendido_acoes = fundo_espec.loc[filt_vendido_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada título público
    porcentagem_vendido = lambda x: (x / df_vendido_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_vendido_acoes['PORCENTAGEM'] = list(map(porcentagem_vendido, df_vendido_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_vendido_acoes = df_vendido_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    return (
        df_acoes, 
        df_bdr, 
        df_exterior, 
        df_cotas_fundos, 
        df_titulos_pub, 
        df_vendido_acoes
    )


def fundo_cnpj_acoes(cnpj: str) -> DataFrame:
    """
    Separando o df do fundo de investimento apenas na categoria de ações.

    Parameters:
    cnpj (str): cnpj do fundo de investimento que você está procurando.

    Returns:
    Vários dataframes de categorias diferentes: ações, BDRs, investimentos no exterior, cotas de fundos e títulos públicos.
    
    """
    # Lendo o df concatenado
    filt_cnpj = df_ativos['CNPJ_FUNDO'] == cnpj
    fundo_espec = df_ativos.loc[filt_cnpj]

    # Ações
    filt_acoes = (fundo_espec['TP_APLIC'] == 'Ações')
    # Selecionando pelo em ordem da maior posição do fundo p/ a menor
    df_acoes = fundo_espec.loc[filt_acoes].sort_values(by='VL_MERC_POS_FINAL', ascending=False)
    # Calculando quantos porcentos representa cada ação
    porcentagem_acao = lambda x: (x / df_acoes['VL_MERC_POS_FINAL'].sum())
    # Criando a coluna 'PORCENTAGEM'
    df_acoes['PORCENTAGEM'] = list(map(porcentagem_acao, df_acoes['VL_MERC_POS_FINAL']))
    # Selecionando apenas as colunas necessárias
    df_acoes = df_acoes.loc[:,['DENOM_SOCIAL', 'CD_ATIVO', 'PORCENTAGEM', 'VL_MERC_POS_FINAL']]

    return df_acoes

In [219]:
# Aplicando as funções para formatar os arquivos excel
df_cda_1 = open_cda_1(path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_1_{ano}{mes}.csv')
df_cda_2 = open_cda_2(path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_2_{ano}{mes}.csv')
df_cda_4 = open_cda_4(path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_4_{ano}{mes}.csv')
df_cda_7 = open_cda_7(path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_7_{ano}{mes}.csv')
df_cda_8 = open_cda_8(path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_8_{ano}{mes}.csv')

# Concatenando os dfs
df_ativos = pd.concat([df_cda_1, df_cda_2, df_cda_4, df_cda_7, df_cda_8])
df_ativos.tail()

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,TP_APLIC,TP_ATIVO,VL_MERC_POS_FINAL,CD_ATIVO
94719,FI,27.293.712/0001-00,CR FUNDO DE INVESTIMENTO MULTIMERCADO CREDITO ...,2023-05-31,Brazilian Depository Receipt - BDR,BDR nível I,89.72,XPBR DR1
116215,FI,34.658.753/0001-00,DAYCOVAL FUNDO DE INVESTIMENTO EM AÇÕES BDR NÍ...,2023-05-31,Brazilian Depository Receipt - BDR,BDR nível I,69.78,MACY DRN
64506,FI,13.089.341/0001-27,FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRI...,2023-05-31,Brazilian Depository Receipt - BDR,BDR nível III,45.00,PPLA11
127930,FI,37.553.253/0001-00,NIMROD FUNDO DE INVESTIMENTO MULTIMERCADO CRÉD...,2023-05-31,Brazilian Depository Receipt - BDR,BDR nível I,25.40,INBR/INBR32/BRINBRBDR007
147554,FI,42.294.675/0001-30,CARTEIRA INTER GLOBAL FUNDO DE INVESTIMENTO EM...,2023-05-31,Brazilian Depository Receipt - BDR,BDR não patrocinado,0.00,DEEC34 BDR


In [18]:
# Quais são os tipos de ativos o fundo de investimento possui
lista_ativos = df_ativos['TP_APLIC'].drop_duplicates().to_list()
lista_ativos

['Operações Compromissadas',
 'Títulos Públicos',
 'Cotas de Fundos',
 'Ações',
 'Debêntures',
 'Certificado ou recibo de depósito de valores mobiliários',
 'Outros valores mobiliários registrados na CVM objeto de oferta pública',
 'Ações e outros TVM cedidos em empréstimo',
 'Brazilian Depository Receipt - BDR',
 'Mercado Futuro - Posições compradas',
 'Mercado Futuro - Posições vendidas',
 'Opções - Posições lançadas',
 'Opções - Posições titulares',
 'Obrigações por ações e outros TVM recebidos em empréstimo',
 'Compras a termo a receber',
 'Vendas a termo a receber',
 'Investimento no Exterior']

## Fundos de Investimentos

### Fundo Verde

* https://maisretorno.com/fundo/verde-am-long-bias-master-fia/carteira

In [82]:
verde_acoes, verde_bdr, verde_exterior, verde_cotas_fundos, verde_titulos_pub, verde_vendido_acoes = fundo_cnpj(cnpj='16.929.553/0001-63')
verde_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.csv', 
    cnpj='16.929.553/0001-63'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(verde_acoes['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_bdr = round(pd.Series(verde_bdr['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_exterior = round(pd.Series(verde_exterior['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(verde_cotas_fundos['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_titulos_pub = round(pd.Series(verde_titulos_pub['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(verde_vendido_acoes['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
verde_acoes = pd.concat([verde_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
verde_acoes['data'] = data
# Definindo a coluna 'data' como o index
verde_acoes = verde_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
verde_bdr = pd.concat([verde_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
verde_bdr['data'] = data
# Definindo a coluna 'data' como o index
verde_bdr = verde_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
verde_exterior = pd.concat([verde_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
verde_exterior['data'] = data
# Definindo a coluna 'data' como o index
verde_exterior = verde_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
verde_cotas_fundos = pd.concat([verde_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
verde_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
verde_cotas_fundos = verde_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
verde_titulos_pub = pd.concat([verde_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
verde_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
verde_titulos_pub = verde_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
verde_vendido_acoes = pd.concat([verde_vendido_acoes, df_linha_pct_vendido_acoes ])
# Criando a coluna 'data' no df
verde_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
verde_veverde_vendido_acoes =verde_vendido_acoes.set_index('data')

In [83]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//verde//fundo_verde_{ano}{mes}.xlsx') as writer:
    verde_acoes.to_excel(writer, sheet_name='acoes')
    verde_bdr.to_excel(writer, sheet_name='bdr')
    verde_exterior.to_excel(writer, sheet_name='exterior')
    verde_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    verde_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    verde_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [10]:
# Abrindo os arquivos excel
arquivos_excel = [
    'fundo_verde_202301.xlsx', 
    'fundo_verde_202302.xlsx', 
    'fundo_verde_202303.xlsx', 
    'fundo_verde_202304.xlsx', 
    'fundo_verde_202305.xlsx'
]  

lst_dfs = []
for arquivo in arquivos_excel:
    df = pd.read_excel(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//verde//{arquivo}', sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_verde = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_verde = portfolio_verde[portfolio_verde['VL_MERC_POS_FINAL'] != 0]
portfolio_verde = portfolio_verde.dropna()

# Definindo a coluna 'data' como o index
portfolio_verde = portfolio_verde.set_index('data')

# Selecionando o conjunto de ações (portfólio) de cada mês
portfolio_mensal = portfolio_verde.groupby('data')['CD_ATIVO'].apply(set)
portfolio_mensal

data
2023-01-01    {ENJU3, RAIZ4, HAPV3, BRKM5, USIM5, MEGA3, RAI...
2023-02-01    {ENJU3, RAIZ4, BRKM5, HAPV3, EZTC3, PSSA3, MEG...
2023-03-01    {ENJU3, HAPV3, BRKM5, PSSA3, MEGA3, RAIL3, VAL...
2023-04-01    {HAPV3, EZTC3, MEGA3, RAIL3, VALE3, NINJ3, BRF...
2023-05-01    {HAPV3, HYPE3, WEGE3, USIM5, MEGA3, RAIL3, VAL...
Name: CD_ATIVO, dtype: object

In [3]:
print('Comparando o portfólio de 01/23 e 02/23:')
print(f'O Verde vendeu as ações: {set(portfolio_mensal[0]) - set(portfolio_mensal[1])}')
print(f'O Verde comprou as ações: {set(portfolio_mensal[1]) - set(portfolio_mensal[0])}')

print('-'*80)

print('Comparando o portfólio de 02/23 e 03/23:')
print(f'O Verde vendeu as ações: {set(portfolio_mensal[1]) - set(portfolio_mensal[2])}')
print(f'O Verde comprou as ações: {set(portfolio_mensal[2]) - set(portfolio_mensal[1])}')

print('-'*80)

print('Comparando o portfólio de 03/23 e 04/23:')
print(f'O Verde vendeu as ações: {set(portfolio_mensal[2]) - set(portfolio_mensal[3])}')
print(f'O Verde comprou as ações: {set(portfolio_mensal[3]) - set(portfolio_mensal[2])}')

print('-'*80)

print('Comparando o portfólio de 04/23 e 05/23:')
print(f'O Verde vendeu as ações: {set(portfolio_mensal[3]) - set(portfolio_mensal[4])}')
print(f'O Verde comprou as ações: {set(portfolio_mensal[4]) - set(portfolio_mensal[3])}')

Comparando o portfólio de 01/23 e 02/23:
O Verde vendeu as ações: {'MULT3', 'CASH3', 'PETR4', 'SOMA3', 'CMIG3', 'RECV3', 'MODL3', 'ALSO3', 'BEEF3', 'TIMS3', 'LJQQ3', 'YDUQ3'}
O Verde comprou as ações: {'EZTC3', 'PSSA3', 'CPLE6', 'VALE3', 'EGIE3', 'MRFG3', 'CRFB3', 'LWSA3', 'BOAS3', 'JBSS3', 'EMBR3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 02/23 e 03/23:
O Verde vendeu as ações: {'RAIZ4', 'GOAU4', 'EZTC3', 'BBAS3', 'GGBR4', 'CSAN3', 'RDOR3', 'GMAT3', 'MRFG3', 'SQIA3', 'CIEL3', 'INTB3', 'MTRE3', 'LREN3', 'EMBR3'}
O Verde comprou as ações: {'SMTO3', 'STBP3', 'ELET6', 'BRFS3', 'DIRR3', 'BEEF3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/23 e 04/23:
O Verde vendeu as ações: {'ENJU3', 'BRKM5', 'RRRP3', 'ELET3', 'USIM5', 'PSSA3', 'BRAP4', 'CBAV3', 'PETZ3', 'BRPR3', 'LWSA3', 'CRFB3', 'DIRR3'}
O Verde comprou as ações: {'ALSO3', 'EZTC3', 'BBAS3', 'GGBR4', 'MRVE3'

In [4]:
# Número total de ações de cada mês do portfpolio
print('Número total de ações:')
num_total_acoes = portfolio_verde.groupby('data')['CD_ATIVO'].count()
print(pd.DataFrame(num_total_acoes))

print('-'*60)
# Selecionando os valores únicos (datas) do df
lst_data = portfolio_verde.index.unique()

# Rank das 5 maiores ações do fundo
lst_rank = []
for _ in lst_data:
    rank = portfolio_verde.loc[_].nlargest(5, 'PORCENTAGEM')[['CD_ATIVO', 'PORCENTAGEM']]
    lst_rank.append(rank)

# Concatendo os dfs de rank
rank_portfolio = pd.concat(lst_rank)
rank_portfolio_verde = rank_portfolio.groupby('data')['CD_ATIVO'].apply(list)
print(rank_portfolio_verde)

Número total de ações:
            CD_ATIVO
data                
2023-01-01        71
2023-02-01        70
2023-03-01        61
2023-04-01        60
2023-05-01        59
------------------------------------------------------------
data
2023-01-01    [SUZB3, RENT3, EQTL3, VBBR3, RAIL3]
2023-02-01    [SUZB3, RENT3, RAIL3, VBBR3, EQTL3]
2023-03-01    [SUZB3, RAIL3, VBBR3, ASAI3, SBSP3]
2023-04-01    [SUZB3, RAIL3, VBBR3, RENT3, EQTL3]
2023-05-01    [SUZB3, RAIL3, EQTL3, VBBR3, SBSP3]
Name: CD_ATIVO, dtype: object


In [96]:
# Plotando o portfólio de cada mês
portfolio_verde_202301 = portfolio_verde.loc['2023-01', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_verde_202302 = portfolio_verde.loc['2023-02', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_verde_202303 = portfolio_verde.loc['2023-03', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_verde_202304 = portfolio_verde.loc['2023-04', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_verde_202305 = portfolio_verde.loc['2023-05', ['CD_ATIVO', 'PORCENTAGEM']]

fig = make_subplots(rows=5,
                    cols=1,
                    subplot_titles=(
                        'Distribuição Percentual do Portfólio - 01/2023',
                        'Distribuição Percentual do Portfólio - 02/2023',
                        'Distribuição Percentual do Portfólio - 03/2023',
                        'Distribuição Percentual do Portfólio - 04/2023',
                        'Distribuição Percentual do Portfólio - 05/2023'),
                    vertical_spacing=0.02 # Espaço entre os plots
)

fig.add_trace(go.Bar(
    x=portfolio_verde_202301['PORCENTAGEM'] * 100,
    y=portfolio_verde_202301['CD_ATIVO'],
    orientation='h',
    name='01/2023' 
), row=1, col=1)

fig.add_trace(go.Bar(
    x=portfolio_verde_202302['PORCENTAGEM'] * 100,
    y=portfolio_verde_202302['CD_ATIVO'],
    orientation='h',
    name='02/2023' 
), row=2, col=1)

fig.add_trace(go.Bar(
    x=portfolio_verde_202303['PORCENTAGEM'] * 100,
    y=portfolio_verde_202303['CD_ATIVO'],
    orientation='h',
    name='03/2023' 
), row=3, col=1)

fig.add_trace(go.Bar(
    x=portfolio_verde_202304['PORCENTAGEM'] * 100,
    y=portfolio_verde_202304['CD_ATIVO'],
    orientation='h',
    name='04/2023' 
), row=4, col=1)

fig.add_trace(go.Bar(
    x=portfolio_verde_202305['PORCENTAGEM'] * 100,
    y=portfolio_verde_202305['CD_ATIVO'],
    orientation='h',
    name='05/2023' 
), row=5, col=1)

# Atualiza o layout do gráfico
fig.update_layout(
    title='Portfólio do Fundo Verde',
    height=6000,
    width=900
)

# Exibe o gráfico
fig.show()

### Fundo Dynamo

* https://maisretorno.com/fundo/dynamo-cougar-master-fia/carteira

In [162]:
dynamo_acoes, dynamo_bdr, dynamo_exterior, dynamo_cotas_fundos, dynamo_titulos_pub, dynamo_vendido_acoes = fundo_cnpj(cnpj='37.916.879/0001-26')
dynamo_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.csv', 
    cnpj='37.916.879/0001-26'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(dynamo_acoes['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_bdr = round(pd.Series(dynamo_bdr['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_exterior = round(pd.Series(dynamo_exterior['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(dynamo_cotas_fundos['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_titulos_pub = round(pd.Series(dynamo_titulos_pub['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(dynamo_vendido_acoes['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
dynamo_acoes = pd.concat([dynamo_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
dynamo_acoes['data'] = data
# Definindo a coluna 'data' como o index
dynamo_acoes = dynamo_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
dynamo_bdr = pd.concat([dynamo_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
dynamo_bdr['data'] = data
# Definindo a coluna 'data' como o index
dynamo_bdr = dynamo_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
dynamo_exterior = pd.concat([dynamo_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
dynamo_exterior['data'] = data
# Definindo a coluna 'data' como o index
dynamo_exterior = dynamo_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
dynamo_cotas_fundos = pd.concat([dynamo_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
dynamo_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
dynamo_codynamo_cotas_fundos =dynamo_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
dynamo_titulos_pub = pd.concat([dynamo_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
dynamo_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
dynamo_titulos_pub = dynamo_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
dynamo_vendido_acoes = pd.concat([dynamo_vendido_acoes, df_linha_pct_vendido_acoes])
# Criando a coluna 'data' no df
dynamo_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
dynamo_vendido_acoes = dynamo_vendido_acoes.set_index('data')

In [163]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//dynamo//fundo_dynamo_{ano}{mes}.xlsx') as writer:
    dynamo_acoes.to_excel(writer, sheet_name='acoes')
    dynamo_bdr.to_excel(writer, sheet_name='bdr')
    dynamo_exterior.to_excel(writer, sheet_name='exterior')
    dynamo_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    dynamo_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    dynamo_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [73]:
# Abrindo os arquivos excel
arquivos_excel = [
    'fundo_dynamo_202301.xlsx', 
    'fundo_dynamo_202302.xlsx', 
    'fundo_dynamo_202303.xlsx', 
    'fundo_dynamo_202304.xlsx', 
    #'fundo_dynamo_202305.xlsx'   #não tem os dados do fundo neste mês, mas tem no site da CVM e Mais Retorno.
]  

lst_dfs = []
for arquivo in arquivos_excel:
    df = pd.read_excel(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//dynamo//{arquivo}', sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_dynamo = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_dynamo = portfolio_dynamo[portfolio_dynamo['VL_MERC_POS_FINAL'] != 0]
portfolio_dynamo = portfolio_dynamo.dropna()

# Definindo a coluna 'data' como o index
portfolio_dynamo = portfolio_dynamo.set_index('data')

# Selecionando o conjunto de ações (portfólio) de cada mês
portfolio_mensal = portfolio_dynamo.groupby('data')['CD_ATIVO'].apply(set)
portfolio_mensal

data
2023-01-01    {ENJU3, WEGE3, PORT3, VALE3, ITSA4, ANIM3, SMF...
2023-02-01    {ENJU3, WEGE3, PORT3, VALE3, ITSA4, ANIM3, SMF...
2023-03-01    {WEGE3, PORT3, VALE3, ITSA4, ANIM3, ASAI3, SMF...
2023-04-01    {WEGE3, PORT3, VALE3, ITSA4, ANIM3, ASAI3, SMF...
Name: CD_ATIVO, dtype: object

In [74]:
print('Comparando o portfólio de 01/23 e 02/23:')
print(f'A Dynamo vendeu as ações: {set(portfolio_mensal[0]) - set(portfolio_mensal[1])}')
print(f'A Dynamo comprou as ações: {set(portfolio_mensal[1]) - set(portfolio_mensal[0])}')

print('-'*80)

print('Comparando o portfólio de 02/23 e 03/23:')
print(f'A Dynamo vendeu as ações: {set(portfolio_mensal[1]) - set(portfolio_mensal[2])}')
print(f'A Dynamo comprou as ações: {set(portfolio_mensal[2]) - set(portfolio_mensal[1])}')

print('-'*80)

print('Comparando o portfólio de 03/23 e 04/23:')
print(f'A Dynamo vendeu as ações: {set(portfolio_mensal[2]) - set(portfolio_mensal[3])}')
print(f'A Dynamo comprou as ações: {set(portfolio_mensal[3]) - set(portfolio_mensal[2])}')


Comparando o portfólio de 01/23 e 02/23:
A Dynamo vendeu as ações: {'ITUB4', 'RENT3', 'PETR3', 'PETR4'}
A Dynamo comprou as ações: {'CCRO3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 02/23 e 03/23:
A Dynamo vendeu as ações: {'ENJU3', 'GGBR4'}
A Dynamo comprou as ações: {'OPCT3', 'ASAI3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/23 e 04/23:
A Dynamo vendeu as ações: set()
A Dynamo comprou as ações: {'GGBR4'}


In [75]:
# Número total de ações de cada mês do portfpolio
print('Número total de ações:')
num_total_acoes = portfolio_dynamo.groupby('data')['CD_ATIVO'].count()
print(pd.DataFrame(num_total_acoes))

print('-'*60)
# Selecionando os valores únicos (datas) do df
lst_data = portfolio_dynamo.index.unique()

# Rank das 5 maiores ações do fundo
lst_rank = []
for _ in lst_data:
    rank = portfolio_dynamo.loc[_].nlargest(5, 'PORCENTAGEM')[['CD_ATIVO', 'PORCENTAGEM']]
    lst_rank.append(rank)

# Concatendo os dfs de rank
rank_portfolio = pd.concat(lst_rank)
rank_portfolio_dynamo = rank_portfolio.groupby('data')['CD_ATIVO'].apply(list)
print(rank_portfolio_dynamo)

Número total de ações:
            CD_ATIVO
data                
2023-01-01        30
2023-02-01        27
2023-03-01        27
2023-04-01        28
------------------------------------------------------------
data
2023-01-01    [ENEV3, VBBR3, CSAN3, RENT3, NTCO3]
2023-02-01    [ENEV3, VBBR3, CSAN3, NTCO3, RDOR3]
2023-03-01    [ENEV3, VBBR3, CSAN3, NTCO3, RDOR3]
2023-04-01    [ENEV3, CSAN3, VBBR3, RDOR3, NTCO3]
Name: CD_ATIVO, dtype: object


In [97]:
# Plotando o portfólio de cada mês
portfolio_dynamo_202301 = portfolio_dynamo.loc['2023-01', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_dynamo_202302 = portfolio_dynamo.loc['2023-02', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_dynamo_202303 = portfolio_dynamo.loc['2023-03', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_dynamo_202304 = portfolio_dynamo.loc['2023-04', ['CD_ATIVO', 'PORCENTAGEM']]
#portfolio_dynamo_202305 = portfolio_dynamo.loc['2023-05', ['CD_ATIVO', 'PORCENTAGEM']]  #não tem os dados do fundo neste mês, mas tem no site da CVM e Mais Retorno.

fig = make_subplots(rows=4,
                    cols=1,
                    subplot_titles=(
                        'Distribuição Percentual do Portfólio - 01/2023',
                        'Distribuição Percentual do Portfólio - 02/2023',
                        'Distribuição Percentual do Portfólio - 03/2023',
                        'Distribuição Percentual do Portfólio - 04/2023'),
                    vertical_spacing=0.03 # Espaço entre os plots
)

fig.add_trace(go.Bar(
    x=portfolio_dynamo_202301['PORCENTAGEM'] * 100,
    y=portfolio_dynamo_202301['CD_ATIVO'],
    orientation='h',
    name='01/2023' 
), row=1, col=1)

fig.add_trace(go.Bar(
    x=portfolio_dynamo_202302['PORCENTAGEM'] * 100,
    y=portfolio_dynamo_202302['CD_ATIVO'],
    orientation='h',
    name='02/2023' 
), row=2, col=1)

fig.add_trace(go.Bar(
    x=portfolio_dynamo_202303['PORCENTAGEM'] * 100,
    y=portfolio_dynamo_202303['CD_ATIVO'],
    orientation='h',
    name='03/2023' 
), row=3, col=1)

fig.add_trace(go.Bar(
    x=portfolio_dynamo_202304['PORCENTAGEM'] * 100,
    y=portfolio_dynamo_202304['CD_ATIVO'],
    orientation='h',
    name='04/2023' 
), row=4, col=1)

# Atualiza o layout do gráfico
fig.update_layout(
    title='Portfólio do Fundo Dynamo',
    height=3000,
    width=900
)

# Exibe o gráfico
fig.show()

### Fundo IP

* https://maisretorno.com/fundo/ip-participacoes-master-fia-bdr-nivel-i/carteira

In [86]:
ip_acoes, ip_bdr, ip_exterior, ip_cotas_fundos, ip_titulos_pub, ip_vendido_acoes = fundo_cnpj(cnpj='11.435.298/0001-89')
ip_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.csv', 
    cnpj='11.435.298/0001-89'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(ip_acoes['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_bdr = round(pd.Series(ip_bdr['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_exterior = round(pd.Series(ip_exterior['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(ip_cotas_fundos['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_titulos_pub = round(pd.Series(ip_titulos_pub['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(ip_vendido_acoes['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
ip_acoes = pd.concat([ip_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
ip_acoes['data'] = data
# Definindo a coluna 'data' como o index
ip_acoes = ip_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
ip_bdr = pd.concat([ip_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
ip_bdr['data'] = data
# Definindo a coluna 'data' como o index
ip_bdr = ip_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
ip_exterior = pd.concat([ip_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
ip_exterior['data'] = data
# Definindo a coluna 'data' como o index
ip_exterior = ip_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
ip_cotas_fundos = pd.concat([ip_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
ip_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
ip_cotas_fundos = ip_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
ip_titulos_pub = pd.concat([ip_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
ip_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
ip_titulos_pub = ip_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
ip_vendido_acoes = pd.concat([ip_vendido_acoes, df_linha_pct_vendido_acoes])
# Criando a coluna 'data' no df
ip_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
ip_vendido_acoes = ip_vendido_acoes.set_index('data')

In [87]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//ip//fundo_ip_{ano}{mes}.xlsx') as writer:
    ip_acoes.to_excel(writer, sheet_name='acoes')
    ip_bdr.to_excel(writer, sheet_name='bdr')
    ip_exterior.to_excel(writer, sheet_name='exterior')
    ip_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    ip_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    ip_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [78]:
# Abrindo os arquivos excel
arquivos_excel = [
    'fundo_ip_202301.xlsx', 
    'fundo_ip_202302.xlsx', 
    'fundo_ip_202303.xlsx', 
    'fundo_ip_202304.xlsx', 
    #'fundo_ip_202305.xlsx'  # não tem os dados do fundo neste mês, mas tem no site da CVM e Mais Retorno.
]  

lst_dfs = []
for arquivo in arquivos_excel:
    df = pd.read_excel(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//ip//{arquivo}', sheet_name='bdr')
    lst_dfs.append(df)

# Concatendo os dfs de bdrs
portfolio_ip = pd.concat(lst_dfs, ignore_index=True)

# Retirando as bdrs que não tem nenhum dinheiro alocado e os NAN
portfolio_ip = portfolio_ip[portfolio_ip['VL_MERC_POS_FINAL'] != 0]
portfolio_ip = portfolio_ip.dropna()

# Definindo a coluna 'data' como o index
portfolio_ip = portfolio_ip.set_index('data')

# Selecionando o conjunto de ações (portfólio) de cada mês
portfolio_mensal = portfolio_ip.groupby('data')['CD_ATIVO'].apply(set)
portfolio_mensal

data
2023-01-01    {GOGL34, XPBR31, T1DG34, AMZO34, NFLX34, MSFT3...
2023-02-01    {GOGL34, XPBR31, T1DG34, AMZO34, NFLX34, BERK3...
2023-03-01    {GOGL34, XPBR31, T1DG34, AMZO34, NFLX34, BERK3...
2023-04-01    {GOGL34, T1DG34, NFLX34, BERK34, MSFT34, LBRD3...
Name: CD_ATIVO, dtype: object

In [79]:
print('Comparando o portfólio de 01/23 e 02/23:')
print(f'A IP vendeu as ações: {set(portfolio_mensal[0]) - set(portfolio_mensal[1])}')
print(f'A IP comprou as ações: {set(portfolio_mensal[1]) - set(portfolio_mensal[0])}')

print('-'*80)

print('Comparando o portfólio de 02/23 e 03/23:')
print(f'A IP vendeu as ações: {set(portfolio_mensal[1]) - set(portfolio_mensal[2])}')
print(f'A IP comprou as ações: {set(portfolio_mensal[2]) - set(portfolio_mensal[1])}')

print('-'*80)

print('Comparando o portfólio de 03/23 e 04/23:')
print(f'A IP vendeu as ações: {set(portfolio_mensal[2]) - set(portfolio_mensal[3])}')
print(f'A IP comprou as ações: {set(portfolio_mensal[3]) - set(portfolio_mensal[2])}')


Comparando o portfólio de 01/23 e 02/23:
A IP vendeu as ações: set()
A IP comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 02/23 e 03/23:
A IP vendeu as ações: set()
A IP comprou as ações: {'MSCD34'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/23 e 04/23:
A IP vendeu as ações: {'AMZO34', 'XPBR31'}
A IP comprou as ações: set()


In [80]:
# Número total de ações de cada mês do portfólio
print('Número total de ações:')
num_total_acoes = portfolio_ip.groupby('data')['CD_ATIVO'].count()
print(pd.DataFrame(num_total_acoes))

print('-'*60)
# Selecionando os valores únicos (datas) do df
lst_data = portfolio_ip.index.unique()

# Rank das 5 maiores ações do fundo
lst_rank = []
for _ in lst_data:
    rank = portfolio_ip.loc[_].nlargest(5, 'PORCENTAGEM')[['CD_ATIVO', 'PORCENTAGEM']]
    lst_rank.append(rank)

# Concatendo os dfs de rank
rank_portfolio = pd.concat(lst_rank)
rank_portfolio_ip = rank_portfolio.groupby('data')['CD_ATIVO'].apply(list)
print(rank_portfolio_ip)

Número total de ações:
            CD_ATIVO
data                
2023-01-01         8
2023-02-01         8
2023-03-01         9
2023-04-01         7
------------------------------------------------------------
data
2023-01-01    [GOGL34, BERK34, T1DG34, LBRD34, MSFT34]
2023-02-01    [BERK34, T1DG34, GOGL34, LBRD34, AMZO34]
2023-03-01    [GOGL34, BERK34, LBRD34, MSCD34, MSFT34]
2023-04-01    [GOGL34, BERK34, LBRD34, MSCD34, MSFT34]
Name: CD_ATIVO, dtype: object


In [92]:
# Plotando o portfólio de cada mês
portfolio_ip_202301 = portfolio_ip.loc['2023-01', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_ip_202302 = portfolio_ip.loc['2023-02', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_ip_202303 = portfolio_ip.loc['2023-03', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_ip_202304 = portfolio_ip.loc['2023-04', ['CD_ATIVO', 'PORCENTAGEM']]
#portfolio_ip_202305 = portfolio_ip.loc['2023-05', ['CD_ATIVO', 'PORCENTAGEM']]  #não tem os dados do fundo neste mês, mas tem no site da CVM e Mais Retorno.

fig = make_subplots(rows=4,
                    cols=1,
                    subplot_titles=(
                        'Distribuição Percentual do Portfólio - 01/2023',
                        'Distribuição Percentual do Portfólio - 02/2023',
                        'Distribuição Percentual do Portfólio - 03/2023',
                        'Distribuição Percentual do Portfólio - 04/2023'),
                    vertical_spacing=0.03 # Espaço entre os plots
)

fig.add_trace(go.Bar(
    x=portfolio_ip_202301['PORCENTAGEM'] * 100,
    y=portfolio_ip_202301['CD_ATIVO'],
    orientation='h',
    name='01/2023' 
), row=1, col=1)

fig.add_trace(go.Bar(
    x=portfolio_ip_202302['PORCENTAGEM'] * 100,
    y=portfolio_ip_202302['CD_ATIVO'],
    orientation='h',
    name='02/2023' 
), row=2, col=1)

fig.add_trace(go.Bar(
    x=portfolio_ip_202303['PORCENTAGEM'] * 100,
    y=portfolio_ip_202303['CD_ATIVO'],
    orientation='h',
    name='03/2023' 
), row=3, col=1)

fig.add_trace(go.Bar(
    x=portfolio_ip_202304['PORCENTAGEM'] * 100,
    y=portfolio_ip_202304['CD_ATIVO'],
    orientation='h',
    name='04/2023' 
), row=4, col=1)

# Atualiza o layout do gráfico
fig.update_layout(
    title='Portfólio do Fundo IP',
    height=2500,
    width=900
)

# Exibe o gráfico
fig.show()

### Fundo Squadra

* https://maisretorno.com/fundo/squadra-master-long-biased-fia
* Famosos pelos shorts de:
    * IRBR - 05/2018
    * AERI3 - 01/2021
    * Nubank

In [88]:
squadra_acoes, squadra_bdr, squadra_exterior, squadra_cotas_fundos, squadra_titulos_pub, squadra_vendido_acoes = fundo_cnpj(cnpj='09.412.648/0001-40')
squadra_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.csv', 
    cnpj='09.412.648/0001-40'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(squadra_acoes['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_bdr = round(pd.Series(squadra_bdr['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_exterior = round(pd.Series(squadra_exterior['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(squadra_cotas_fundos['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_titulos_pub = round(pd.Series(squadra_titulos_pub['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(squadra_vendido_acoes['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
squadra_acoes = pd.concat([squadra_acoes, df_linha_pct_acoes], ignore_index=True)
# Criando a coluna 'data' no df
squadra_acoes['data'] = data
# Definindo a coluna 'data' como o index
squadra_acoes = squadra_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
squadra_bdr = pd.concat([squadra_bdr, df_linha_pct_bdr], ignore_index=True)
# Criando a coluna 'data' no df
squadra_bdr['data'] = data
# Definindo a coluna 'data' como o index
squadra_bdr = squadra_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
squadra_exterior = pd.concat([squadra_exterior, df_linha_pct_exterior], ignore_index=True)
# Criando a coluna 'data' no df
squadra_exterior['data'] = data
# Definindo a coluna 'data' como o index
squadra_exterior = squadra_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
squadra_cotas_fundos = pd.concat([squadra_cotas_fundos, df_linha_pct_cotas_fundos], ignore_index=True)
# Criando a coluna 'data' no df
squadra_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
squadra_cotas_fundos = squadra_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
squadra_titulos_pub = pd.concat([squadra_titulos_pub, df_linha_pct_titulos_pub], ignore_index=True)
# Criando a coluna 'data' no df
squadra_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
squadra_titulos_pub = squadra_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
squadra_vendido_acoes = pd.concat([squadra_vendido_acoes, df_linha_pct_vendido_acoes], ignore_index=True)
# Criando a coluna 'data' no df
squadra_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
squadra_vendido_acoes = squadra_vendido_acoes.set_index('data')

In [89]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//squadra//fundo_squadra_{ano}{mes}.xlsx') as writer:
    squadra_acoes.to_excel(writer, sheet_name='acoes')
    squadra_bdr.to_excel(writer, sheet_name='bdr')
    squadra_exterior.to_excel(writer, sheet_name='exterior')
    squadra_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    squadra_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    squadra_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [85]:
# Abrindo os arquivos excel
arquivos_excel = [
    'fundo_squadra_202301.xlsx', 
    'fundo_squadra_202302.xlsx', 
    'fundo_squadra_202303.xlsx', 
    'fundo_squadra_202304.xlsx', 
    'fundo_squadra_202305.xlsx'
]  

lst_dfs = []
for arquivo in arquivos_excel:
    df = pd.read_excel(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//squadra//{arquivo}', sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_squadra = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_squadra = portfolio_squadra[portfolio_squadra['VL_MERC_POS_FINAL'] != 0]
portfolio_squadra = portfolio_squadra.dropna()

# Definindo a coluna 'data' como o index
portfolio_squadra = portfolio_squadra.set_index('data')

# Selecionando o conjunto de ações (portfólio) de cada mês
portfolio_mensal = portfolio_squadra.groupby('data')['CD_ATIVO'].apply(set)
portfolio_mensal

data
2023-01-01    {HAPV3, RAIL3, LWSA3, CRDE3, B3SA3, MULT3, ABE...
2023-02-01    {HAPV3, RAIL3, PETZ3, LWSA3, INTB3, MGLU3, CRD...
2023-03-01    {HAPV3, WEGE3, RAIL3, LWSA3, INTB3, MGLU3, CRD...
2023-04-01    {HAPV3, WEGE3, RAIL3, CXSE3, PETZ3, INTB3, MGL...
2023-05-01    {HAPV3, WEGE3, RAIL3, PETZ3, INTB3, MGLU3, RDO...
Name: CD_ATIVO, dtype: object

In [86]:
print('Comparando o portfólio de 01/23 e 02/23:')
print(f'A Squadra vendeu as ações: {set(portfolio_mensal[0]) - set(portfolio_mensal[1])}')
print(f'A Squadra comprou as ações: {set(portfolio_mensal[1]) - set(portfolio_mensal[0])}')

print('-'*80)

print('Comparando o portfólio de 02/23 e 03/23:')
print(f'A Squadra vendeu as ações: {set(portfolio_mensal[1]) - set(portfolio_mensal[2])}')
print(f'A Squadra comprou as ações: {set(portfolio_mensal[2]) - set(portfolio_mensal[1])}')

print('-'*80)

print('Comparando o portfólio de 03/23 e 04/23:')
print(f'A Squadra vendeu as ações: {set(portfolio_mensal[2]) - set(portfolio_mensal[3])}')
print(f'A Squadra comprou as ações: {set(portfolio_mensal[3]) - set(portfolio_mensal[2])}')

print('-'*80)

print('Comparando o portfólio de 04/23 e 05/23:')
print(f'A Squadra vendeu as ações: {set(portfolio_mensal[3]) - set(portfolio_mensal[4])}')
print(f'A Squadra comprou as ações: {set(portfolio_mensal[4]) - set(portfolio_mensal[3])}')

Comparando o portfólio de 01/23 e 02/23:
A Squadra vendeu as ações: set()
A Squadra comprou as ações: {'MGLU3', 'VIIA3', 'INTB3', 'PETZ3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 02/23 e 03/23:
A Squadra vendeu as ações: {'ENEV3', 'B3SA3', 'PETZ3'}
A Squadra comprou as ações: {'WEGE3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/23 e 04/23:
A Squadra vendeu as ações: {'LWSA3'}
A Squadra comprou as ações: {'CXSE3', 'PETZ3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 04/23 e 05/23:
A Squadra vendeu as ações: {'ITUB4', 'CXSE3', 'MULT3', 'CRFB3'}
A Squadra comprou as ações: {'RDOR3'}


In [87]:
# Número total de ações de cada mês do portfpolio
print('Número total de ações:')
num_total_acoes = portfolio_squadra.groupby('data')['CD_ATIVO'].count()
print(pd.DataFrame(num_total_acoes))

print('-'*60)
# Selecionando os valores únicos (datas) do df
lst_data = portfolio_squadra.index.unique()

# Rank das 5 maiores ações do fundo
lst_rank = []
for _ in lst_data:
    rank = portfolio_squadra.loc[_].nlargest(5, 'PORCENTAGEM')[['CD_ATIVO', 'PORCENTAGEM']]
    lst_rank.append(rank)

# Concatendo os dfs de rank 
rank_portfolio = pd.concat(lst_rank)
rank_portfolio_squadra = rank_portfolio.groupby('data')['CD_ATIVO'].apply(list)
print(rank_portfolio_squadra)

Número total de ações:
            CD_ATIVO
data                
2023-01-01        21
2023-02-01        25
2023-03-01        23
2023-04-01        24
2023-05-01        21
------------------------------------------------------------
data
2023-01-01    [EQTL3, PRIO3, RAIL3, GMAT3, GGPS3]
2023-02-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-03-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-04-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-05-01    [EQTL3, RAIL3, GMAT3, PRIO3, GGPS3]
Name: CD_ATIVO, dtype: object


In [90]:
# Plotando o portfólio de cada mês
portfolio_squadra_202301 = portfolio_squadra.loc['2023-01', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_squadra_202302 = portfolio_squadra.loc['2023-02', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_squadra_202303 = portfolio_squadra.loc['2023-03', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_squadra_202304 = portfolio_squadra.loc['2023-04', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_squadra_202305 = portfolio_squadra.loc['2023-05', ['CD_ATIVO', 'PORCENTAGEM']]

fig = make_subplots(rows=5,
                    cols=1,
                    subplot_titles=(
                        'Distribuição Percentual do Portfólio - 01/2023',
                        'Distribuição Percentual do Portfólio - 02/2023',
                        'Distribuição Percentual do Portfólio - 03/2023',
                        'Distribuição Percentual do Portfólio - 04/2023',
                        'Distribuição Percentual do Portfólio - 05/2023'),
                    vertical_spacing=0.03 # Espaço entre os plots
)

fig.add_trace(go.Bar(
    x=portfolio_squadra_202301['PORCENTAGEM'] * 100,
    y=portfolio_squadra_202301['CD_ATIVO'],
    orientation='h',
    name='01/2023' 
), row=1, col=1)

fig.add_trace(go.Bar(
    x=portfolio_squadra_202302['PORCENTAGEM'] * 100,
    y=portfolio_squadra_202302['CD_ATIVO'],
    orientation='h',
    name='02/2023' 
), row=2, col=1)

fig.add_trace(go.Bar(
    x=portfolio_squadra_202303['PORCENTAGEM'] * 100,
    y=portfolio_squadra_202303['CD_ATIVO'],
    orientation='h',
    name='03/2023' 
), row=3, col=1)

fig.add_trace(go.Bar(
    x=portfolio_squadra_202304['PORCENTAGEM'] * 100,
    y=portfolio_squadra_202304['CD_ATIVO'],
    orientation='h',
    name='04/2023' 
), row=4, col=1)

fig.add_trace(go.Bar(
    x=portfolio_squadra_202305['PORCENTAGEM'] * 100,
    y=portfolio_squadra_202305['CD_ATIVO'],
    orientation='h',
    name='05/2023' 
), row=5, col=1)

# Atualiza o layout do gráfico
fig.update_layout(
    title='Portfólio do Fundo Squadra',
    height=2500,
    width=900
)

# Exibe o gráfico
fig.show()

### Fundo Guepardo

* https://maisretorno.com/fundo/guepardo-institucional-master-fia

In [220]:
guepardo_acoes = fundo_cnpj_acoes(cnpj='14.213.077/0001-54')
guepardo_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.csv', 
    cnpj='14.213.077/0001-54'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(guepardo_acoes['VL_MERC_POS_FINAL'].sum() / guepardo_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
guepardo_acoes = pd.concat([guepardo_acoes, df_linha_pct_acoes], ignore_index=True)
# Criando a coluna 'data' no df
guepardo_acoes['data'] = data
# Definindo a coluna 'data' como o index
guepardo_acoes = guepardo_acoes.set_index('data')

In [221]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//guepardo//fundo_guepardo_{ano}{mes}.xlsx') as writer:
    guepardo_acoes.to_excel(writer, sheet_name='acoes')
    

In [15]:
# Abrindo os arquivos excel
arquivos_excel = [
    'fundo_guepardo_202301.xlsx', 
    'fundo_guepardo_202302.xlsx', 
    'fundo_guepardo_202303.xlsx', 
    'fundo_guepardo_202304.xlsx', 
    'fundo_guepardo_202305.xlsx'
]  

lst_dfs = []
for arquivo in arquivos_excel:
    df = pd.read_excel(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//arquivos_fundos//guepardo//{arquivo}', sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_guepardo = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado  e os NAN
portfolio_guepardo = portfolio_guepardo[portfolio_guepardo['VL_MERC_POS_FINAL'] != 0]
portfolio_guepardo = portfolio_guepardo.dropna()

# Definindo a coluna 'data' como o index
portfolio_guepardo = portfolio_guepardo.set_index('data')

# Selecionando o conjunto de ações (portfólio) de cada mês
portfolio_mensal = portfolio_guepardo.groupby('data')['CD_ATIVO'].apply(set)
portfolio_mensal

data
2023-01-01    {UGPA3, MLAS3, VULC3, MYPK3, RAIL3, GMAT3, ITU...
2023-02-01    {UGPA3, MLAS3, VULC3, MYPK3, RAIL3, GMAT3, ITU...
2023-03-01    {UGPA3, MLAS3, VULC3, MYPK3, RAIL3, GMAT3, ITU...
2023-04-01    {UGPA3, MLAS3, VULC3, MYPK3, RAIL3, GMAT3, ITU...
2023-05-01    {UGPA3, MLAS3, VULC3, GGBR4, MYPK3, RAIL3, GMA...
Name: CD_ATIVO, dtype: object

In [223]:
print('Comparando o portfólio de 01/23 e 02/23:')
print(f'O Guepardo vendeu as ações: {set(portfolio_mensal[0]) - set(portfolio_mensal[1])}')
print(f'O Guepardo comprou as ações: {set(portfolio_mensal[1]) - set(portfolio_mensal[0])}')

print('-'*80)

print('Comparando o portfólio de 02/23 e 03/23:')
print(f'O Guepardo vendeu as ações: {set(portfolio_mensal[1]) - set(portfolio_mensal[2])}')
print(f'O Guepardo comprou as ações: {set(portfolio_mensal[2]) - set(portfolio_mensal[1])}')

print('-'*80)

print('Comparando o portfólio de 03/23 e 04/23:')
print(f'O Guepardo vendeu as ações: {set(portfolio_mensal[2]) - set(portfolio_mensal[3])}')
print(f'O Guepardo comprou as ações: {set(portfolio_mensal[3]) - set(portfolio_mensal[2])}')

print('-'*80)

print('Comparando o portfólio de 04/23 e 05/23:')
print(f'O Guepardo vendeu as ações: {set(portfolio_mensal[3]) - set(portfolio_mensal[4])}')
print(f'O Guepardo comprou as ações: {set(portfolio_mensal[4]) - set(portfolio_mensal[3])}')

Comparando o portfólio de 01/23 e 02/23:
O Guepardo vendeu as ações: set()
O Guepardo comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 02/23 e 03/23:
O Guepardo vendeu as ações: set()
O Guepardo comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 03/23 e 04/23:
O Guepardo vendeu as ações: set()
O Guepardo comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 04/23 e 05/23:
O Guepardo vendeu as ações: {'DXCO3'}
O Guepardo comprou as ações: {'GGBR4'}


In [224]:
# Número total de ações de cada mês do portfpolio
print('Número total de ações:')
num_total_acoes = portfolio_guepardo.groupby('data')['CD_ATIVO'].count()
print(pd.DataFrame(num_total_acoes))

print('-'*60)
# Selecionando os valores únicos (datas) do df
lst_data = portfolio_guepardo.index.unique()

# Rank das 5 maiores ações do fundo
lst_rank = []
for _ in lst_data:
    rank = portfolio_guepardo.loc[_].nlargest(5, 'PORCENTAGEM')[['CD_ATIVO', 'PORCENTAGEM']]
    lst_rank.append(rank)

# Concatendo os dfs de rank
rank_portfolio = pd.concat(lst_rank)
rank_portfolio_guepardo = rank_portfolio.groupby('data')['CD_ATIVO'].apply(list)
print(rank_portfolio_guepardo)

Número total de ações:
            CD_ATIVO
data                
2023-01-01        11
2023-02-01        11
2023-03-01        11
2023-04-01        11
2023-05-01        11
------------------------------------------------------------
data
2023-01-01    [VULC3, ITUB4, UGPA3, RAIL3, ANIM3]
2023-02-01    [VULC3, ITUB4, UGPA3, RAIL3, ANIM3]
2023-03-01    [VULC3, UGPA3, ITUB4, RAIL3, ANIM3]
2023-04-01    [VULC3, UGPA3, ITUB4, RAIL3, ANIM3]
2023-05-01    [VULC3, UGPA3, ITUB4, RAIL3, ANIM3]
Name: CD_ATIVO, dtype: object


In [98]:
# Plotando o portfólio de cada mês
portfolio_guepardo_202301 = portfolio_guepardo.loc['2023-01', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_guepardo_202302 = portfolio_guepardo.loc['2023-02', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_guepardo_202303 = portfolio_guepardo.loc['2023-03', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_guepardo_202304 = portfolio_guepardo.loc['2023-04', ['CD_ATIVO', 'PORCENTAGEM']]
portfolio_guepardo_202305 = portfolio_guepardo.loc['2023-05', ['CD_ATIVO', 'PORCENTAGEM']]

fig = make_subplots(rows=5,
                    cols=1,
                    subplot_titles=(
                        'Distribuição Percentual do Portfólio - 01/2023',
                        'Distribuição Percentual do Portfólio - 02/2023',
                        'Distribuição Percentual do Portfólio - 03/2023',
                        'Distribuição Percentual do Portfólio - 04/2023',
                        'Distribuição Percentual do Portfólio - 05/2023'),
                    vertical_spacing=0.03 # Espaço entre os plots
)

fig.add_trace(go.Bar(
    x=portfolio_guepardo_202301['PORCENTAGEM'] * 100,
    y=portfolio_guepardo_202301['CD_ATIVO'],
    orientation='h',
    name='01/2023' 
), row=1, col=1)

fig.add_trace(go.Bar(
    x=portfolio_guepardo_202302['PORCENTAGEM'] * 100,
    y=portfolio_guepardo_202302['CD_ATIVO'],
    orientation='h',
    name='02/2023' 
), row=2, col=1)

fig.add_trace(go.Bar(
    x=portfolio_guepardo_202303['PORCENTAGEM'] * 100,
    y=portfolio_guepardo_202303['CD_ATIVO'],
    orientation='h',
    name='03/2023' 
), row=3, col=1)

fig.add_trace(go.Bar(
    x=portfolio_guepardo_202304['PORCENTAGEM'] * 100,
    y=portfolio_guepardo_202304['CD_ATIVO'],
    orientation='h',
    name='04/2023' 
), row=4, col=1)

fig.add_trace(go.Bar(
    x=portfolio_guepardo_202305['PORCENTAGEM'] * 100,
    y=portfolio_guepardo_202305['CD_ATIVO'],
    orientation='h',
    name='05/2023' 
), row=5, col=1)

# Atualiza o layout do gráfico
fig.update_layout(
    title='Portfólio do Fundo Guepardo',
    height=2500,
    width=900
)

# Exibe o gráfico
fig.show()